# Triage at home
A notebook demonstrating the data pipeline, using NLP capabilities such as NLTK in the domain of primary health care.
TODO: Finish of with some machine learning.

## Connect to database
Fetches appropriate _code_ and _shorttitle_ from dbase and puts in a datascience.table

In [8]:
# coding: utf-8
import codecs
from datascience import * # pip install datascience # documentation at github.com/data-8/datascience and inferentialthinking.com

import sqlite3
from sqlite3 import Error


def create_connection(db_file):
    """ create a database connection to the SQLite database
        specified by the db_file
    :param db_file: database file
    :return: Connection object or None
    """
    try:
        conn = sqlite3.connect(db_file)
        return conn
    except Error as e:
        print(e)
 
    return None

def select_all_tasks(conn):
    """
    Query all rows in the tasks table
    :param conn: the Connection object
    :return:
    """
    cur = conn.cursor()
    cur.execute("SELECT code,consider,criteria,exclusion,inclusion,note,shorttitle,shorttitle_se FROM icpc")
    # cur.execute("SELECT code FROM icpc WHERE consider=?", (priority,))
 
    rows = cur.fetchall()
 
    for row in rows:
        # print(row)
        shorttitle_se.append(row[7])
        code.append(row[0])
        #print(row[0])

database = "resources/mldb.db"
code = []
shorttitle_se = []

# create a database connection
conn = create_connection(database)
with conn:
    select_all_tasks(conn)
conn.close()

icpc_table = Table().with_columns('Code', code, 'Title (Swedish)', shorttitle_se)
icpc_table

Code,Title (Swedish)
-30,Medicinsk undersökning / grundlig
-31,Medicinsk undersökning / partiell
-32,Allergitest
-33,Mikrobiologiskt /annat immunologiskt test
-34,Blodprov
-35,Urinprov
-36,Avföringsprov
-37,Histologiskt / cytologiskt prov
-38,Annat laboratorieprov UNS
-39,Kliniskt funktionstest


## Processing the strings into words
Removing stopwords and stemming words to their base form.

In [9]:
import nltk
from nltk.stem.snowball import SwedishStemmer
import re

def remove_stopwords(s):
    stopwords = set(w.rstrip() for w in open('resources/stopwords_se.txt'))

    s = s.lower() # downcase
    tokens = nltk.tokenize.word_tokenize(s) # split string into words (tokens)
    tokens = [t for t in tokens if len(t) > 2] # remove short words, they're probably not useful
    # tokens = [wordnet_lemmatizer.lemmatize(t) for t in tokens] # put words into base form - don't work in Swedish though
    tokens = [t for t in tokens if t not in stopwords] # remove stopwords
    return tokens

swedish_stemmer = SwedishStemmer()	#(ignore_stopwords=False)

i = 0 # counter for debugging purposes
stemmed_keywords = []

for item in icpc_table.column(1):
    item = item.replace(',', " ").replace('/', " ").replace('-', ' ')
    item = re.sub(' +',' ',item)
    
    item_keywords = ''
    
    for word in item.split(' '):
        stemmed = swedish_stemmer.stem(word)
        #print(i, remove_stopwords(stemmed), stemmed, word)
        if len(remove_stopwords(stemmed)) > 0:
            #print(remove_stopwords(stemmed))
            item_keywords = '{0}, {1}'.format(item_keywords, remove_stopwords(stemmed)[0])
    
    i += 1
    stemmed_keywords.append(item_keywords[2:])
    #print(i, item_keywords)

icpc_table = icpc_table.with_column('Keywords (stemmed)', stemmed_keywords)
icpc_table

Code,Title (Swedish),Keywords (stemmed)
-30,Medicinsk undersökning / grundlig,"medicinsk, undersökning, grund"
-31,Medicinsk undersökning / partiell,"medicinsk, undersökning, partiell"
-32,Allergitest,allergitest
-33,Mikrobiologiskt /annat immunologiskt test,"mikrobiologisk, ann, immunologisk, test"
-34,Blodprov,blodprov
-35,Urinprov,urinprov
-36,Avföringsprov,avföringsprov
-37,Histologiskt / cytologiskt prov,"histologisk, cytologisk, prov"
-38,Annat laboratorieprov UNS,"ann, laboratorieprov"
-39,Kliniskt funktionstest,"klinisk, funktionstest"


## Medical history as input
Matching the stemmed words from ICPC's titles with stemmed words from the user's medical history (anamnes in Swedish)

In [10]:
s = "Haft hosta under den senaste tiden. Är rökare och har känt sig snuvig i faktiskt flera månader och har även misstanke om någon form av allergi."
s = remove_stopwords(s)

tags = nltk.pos_tag(s)
# print(tags)
print(nltk.ne_chunk(tags))

(S
  hosta/NN
  tiden/NN
  rökare/NN
  känt/FW
  snuvig/NN
  faktiskt/NN
  månader/NN
  misstanke/NN
  allergi/NN)


## Match user input with stemmed keywords from ICPC vocabulary

In [11]:
for word in s:
    stemmed_word = remove_stopwords(swedish_stemmer.stem(word))[0]
    matches = icpc_table.where('Keywords (stemmed)', are.containing(stemmed_word)).column(0)
    
    if len(matches) > 0: # only presenting words with matches in ICPC
        print("The stemmed word '{0}' matched the following codes from ICPC '{1}'".format(stemmed_word, ', '.join(matches)))

The stemmed word 'host' matched the following codes from ICPC 'R05, R24, R25, R71'
The stemmed word 'snuv' matched the following codes from ICPC 'R97'
The stemmed word 'allergi' matched the following codes from ICPC '-32, A92, F71, R97, S88'
